# Analysis of Neighborhoods in Toronto
### K-Clustering of Toronto Neighborhoods based on neighborhood data pulled from Foursquare API

# Introduction

This program serves as an initial, exploratory study of neighborhoods in Toronto in order to gain preliminary insights into various areas of the city.

# Business Case
As a result of this application, we will be able to differentiate each neighborhood in Toronto based on common nearby venues. 

This information would be valuable for marketing strategies in determining what areas to target for certain products or services to be most effective.   
In addition, this data could serve as a starting point for entrepreneurs and investors looking for a location to open a new business.
Moreover, general neighborhood information would be incredibly useful for people hoping to move and are interested in what certain areas
are like. Apartment and housing sites may want to apply this information to their own systems to better sell locations to their clients.

# Methodology
This program was inspired by an MOOC project and involves the following:  
* Extracting List of Toronto Neighborhoods from Wikipedia
* Appending geo data according to an excel file containing geospatial data
* Mapping Toronto Neighborhoods using Folium
* Integrating data of nearby venues from the Foursquare API
* Clustering Neighborhoods according to nearby venues using k-means clustering
* Analysis Neighborhood clusters


In [46]:
import pandas as pd
import numpy as np

## Extract List of Toronto Neighborhoods from Wikipedia

In [47]:
import requests
wikipedia_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_wikipedia_page = requests.get(wikipedia_link).text

In [48]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'lxml')
Toronto_data = BeautifulSoup(raw_wikipedia_page, 'lxml')

In [49]:
# create a new dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

# loop through webpage data to find postcode, borough, neighborhood 
content = Toronto_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

# fill in dataframe with relevant values
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# drop empty boroughs
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)

# if neighborhood is empty, use borough name as neighborhood name
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df_toronto = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Append Longitude and Latitude Data to Neighborhoods

In [50]:
import csv
import json
import xml

In [51]:
'''
Source for geo data file provided by MOOC
'''
filename = "http://cocl.us/Geospatial_data"
headers = ["Postal Code","Latitude","Longitude"]

df_geo = pd.read_csv(filename, names = headers)
df_geo.head()

,Postal Code,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [52]:
# rename columns
df_geo.rename(columns = {'Postal Code':'Postalcode'}, inplace = True)
df_geo.head()

,Postalcode,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [53]:
# join to original neighborhood on postal code
df_map=pd.merge(df_toronto,df_geo,on="Postalcode")
df_map.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


## Map data

In [54]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [55]:
# get geographical data of Toronto for map
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [56]:
# recast latitude and longitude in dataframe for mapping
df_map["Latitude"] = pd.to_numeric(df_map["Latitude"])
df_map["Longitude"] = pd.to_numeric(df_map["Longitude"])

In [57]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_map['Latitude'], df_map['Longitude'], df_map['Borough'], df_map['Neighborhood']):

    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Incorporate Neighborhood Information using Foursquare API

In [58]:
CLIENT_ID = 'O4WM3K3DDF3PCM1ZQLDP414LXUBE3I2INHEVLMI53HKQZJOG' 
CLIENT_SECRET = '3WW3EJJ5DEDMZG11KZXGRPTD51OX50OWVZFJI5C5E4D2SBAA' 
VERSION = '20190330' # Foursquare API version

In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    '''
    Source: MOOC - API
    '''
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [115]:
# get list of 100 nearby venues for each neighborhood
toronto_venues = getNearbyVenues(names=df_map['Neighborhood'],
                                   latitudes=df_map['Latitude'],
                                   longitudes=df_map['Longitude']
                                  )

toronto_venues.head()

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [130]:
# one hot encoding to get count of venue types in each neighborhood
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, 

In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
# Create a dataframe of all neighborhoods and the most common venue types in each

num_top_venues = 5
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # e.g. 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) # e.g. 4th, 5th, 6th, ...

# create a new dataframe consisting of neighborhood and its top 5 most common nearby venues
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# fill in data for Most Common Venue columns
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Yoga Studio
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink


## Cluster Neighborhoods using K-Means Clustering

In [133]:
# set number of clusters
kclusters = 5

# drop neighborhood name in order to cluster purely on the venues
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [134]:
# calculate clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = df_map
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(axis=0, inplace = True)
toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Electronics Store,Rental Car Location,Pizza Place,Spa
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Drugstore,Diner,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Thai Restaurant


In [135]:
# cast cluster labels as int
CL_int = toronto_merged[['Cluster Labels']].astype(int)

toronto_merged = toronto_merged.drop('Cluster Labels', 1)
toronto_merged.insert(0, 'Cluster Labels', CL_int)

toronto_merged.head()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Donut Shop
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Intersection,Electronics Store,Rental Car Location,Pizza Place,Spa
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Drugstore,Diner,Discount Store
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Thai Restaurant


## Map Clusters

In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# set color scheme for the clusters
'''
Source: MOOC - Folium
'''
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster Analysis

In [137]:
# Investigate the first cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Postalcode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Donut Shop
2,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Intersection,Electronics Store,Rental Car Location,Pizza Place,Spa
3,M1G,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Drugstore,Diner,Discount Store
4,M1H,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Thai Restaurant
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Department Store,Discount Store,Bus Station,Coffee Shop,Chinese Restaurant
7,M1L,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Bakery,Bus Line,Fast Food Restaurant,Metro Station,Intersection
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,American Restaurant,Motel,Dim Sum Restaurant,Diner,Discount Store
9,M1N,"Birch Cliff, Cliffside West",43.692657,-79.264848,Café,College Stadium,Skating Rink,General Entertainment,Dog Run
10,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant


In [138]:
# Investigate the second cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Postalcode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,M1J,Scarborough Village,43.744734,-79.239476,Playground,Construction & Landscaping,Yoga Studio,Donut Shop,Dim Sum Restaurant
48,M4T,"Moore Park, Summerhill East",43.689574,-79.383160,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner


In [139]:
# Investigate the third cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Postalcode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,Park,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant
23,M2P,York Mills West,43.752758,-79.400049,Park,Bank,Bar,Yoga Studio,Donut Shop
25,M3A,Parkwoods,43.753259,-79.329656,Fast Food Restaurant,Food & Drink Shop,Park,Dim Sum Restaurant,Diner
30,M3K,"CFB Toronto, Downsview East",43.737473,-79.464763,Park,Airport,Bus Stop,Yoga Studio,Donut Shop
40,M4J,East Toronto,43.685347,-79.338106,Park,Convenience Store,Coffee Shop,Drugstore,Diner
44,M4N,Lawrence Park,43.728020,-79.388790,Park,Swim School,Bus Line,Doner Restaurant,Diner
50,M4W,Rosedale,43.679563,-79.377529,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant
64,M5P,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Park,Trail,Sushi Restaurant,Jewelry Store,Dim Sum Restaurant
74,M6E,Caledonia-Fairbanks,43.689026,-79.453512,Park,Pharmacy,Fast Food Restaurant,Women's Store,Airport
79,M6L,"Downsview, North Park, Upwood Park",43.713756,-79.490074,Park,Bakery,Construction & Landscaping,Basketball Court,Yoga Studio


In [140]:
# Investigate the fourth cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Postalcode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,M2K,Bayview Village,43.786947,-79.385975,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dim Sum Restaurant
94,M9B,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,Bank,Yoga Studio,Drugstore,Discount Store,Dive Bar


In [141]:
# Investigate the fifth cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Postalcode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,M2L,"Silver Hills, York Mills",43.75749,-79.374714,Cafeteria,Donut Shop,Dim Sum Restaurant,Diner,Discount Store


# Conclusion
Toronto Neighborhoods seem to best clustered into groups based on the following venues:  
1. Coffee Shop/Cafe
2. Playground
3. Park
4. Bank
5. Cafeteria

## Potential Business Insights
Cluster 1 consist of neighborhoods with highly social venues such as coffee shops and bars. There are also a significant number of hotels, which suggests a high tourist population. I would expect these place to be the livelier areas of Toronto, most popular amongst adults ages 21-40. These neighborhoods would be the best places to open a new business.

Cluster 2 most likely contains neighborhoods with high family populations. With playgrounds being the most common venues, I would expect these areas to have a much younger demographic of ages 5-15 as well as the parent demographic. This are would likely be attractive to couples looking to settle down.

Cluster 3 refers to nature areas with many outdoor venues such as Parks and Trails. I would not recommend these areas for opening a new business as these areas are likely to be far removed from downtown/urban areas and will have lower foot traffic.

Cluster 4 may be a good area for advertising related financial services.

Cluster 5 seems to be an outlier.

## Areas For Further Investigation
When increasing the number of clusters overall in an attempt to break out all areas into more distinguished categories, Cluster 1 was remained overly large relative to the other clusters, which suggests that the strong presence of Coffee Shops and Cafes may be overwhelming the other venues. As such, it may serve to create sub-clusters solely within the population of Cluster 1 in order to further distinguish these neighborhoods from each other for additional detail. Moreover, dropping Coffee Shops and Cafes from the consideration of sub-clusters as well as incorporating additional venues beyond the top 5 may be effective.

It would be beneficial to incorporate demographic data by neighborhood to confirm the above predictions.

In addition, looking at traffic activity throughout the day and week may provide useful information regarding activity levels, which would provide further information for areas that foster new/existing enterprises.